In [2]:
import gurobipy as gb
import pandas as pd
import gurobipy as gb

In [3]:
df = pd.read_csv("~/Desktop/uni BAM/MS/Group Assignment/flights-1.csv",  encoding='latin-1')

issues = {"ZÃ¼rich": "Zurich",
          "DÃ¼sseldorf": "Dusseldorf", "MÃ¡laga": "Malaga"}
for key, value in issues.items():
    df.loc[df["departureCity"] == key, "departureCity"] = value
    df.loc[df["arrivalCity"] == key, "arrivalCity"] = value

# Joining the DataFrame with itself on the departure city
df2 = df.set_index('departureCity').join(df.set_index('departureCity'), lsuffix="first", rsuffix="second").reset_index()

# Filtering out rows where the arrival cities match
df2 = df2[df2.arrivalCityfirst != df2.arrivalCitysecond]

# Calculating the total distance as the sum of the distances from departure to hub and hub to arrival
df2["distance"] = df2.Distancefirst + df2.Distancesecond

# Selecting relevant columns
df2 = df2[["departureCity", "arrivalCityfirst", "arrivalCitysecond", "distance"]]

# Renaming columns
df2.columns = ["Hub", "arrivalCity", "departureCity", "Distance"]


merged_df = df2.merge(df[["departureCity", "arrivalCity", "Distance", "Demand"]],
               on=["departureCity", "arrivalCity"],
               how="left")

merged_df.columns = ["Hub", "arrivalCity", "departureCity", "IndirectDistance","DirectDistance", "Demand"]

merged_df["1.3times_Direct"] = 1.3 * merged_df["DirectDistance"]
merged_df = merged_df[["Hub", "arrivalCity", "departureCity", "IndirectDistance", "DirectDistance", "1.3times_Direct", "Demand"]]

merged_df.loc[merged_df["IndirectDistance"] >= merged_df["DirectDistance"] * 1.3, "Hub"] = ""
merged_df["RouteDistance"] = merged_df["DirectDistance"]

mask = merged_df["Hub"] != ""
merged_df.loc[mask, "RouteDistance"] = merged_df.loc[mask, "IndirectDistance"]
merged_df["IndirectUsed"] = (merged_df["Hub"] != "").astype(int) # Presence of a hub
merged_df = merged_df.drop_duplicates(subset = ["arrivalCity", "departureCity", "RouteDistance"])


flights=[]
for index,row in merged_df.iterrows():
    if row.Hub=="":
        flights.append((row.departureCity,row.arrivalCity))
    else:
        flights.append((row.departureCity,row.Hub))
        flights.append((row.Hub,row.departureCity))
flights=list(set(flights))#removing duplicates

ItineraryFlights = {}
for index, row in merged_df.iterrows():
    itflights = []
    if row.Hub == "":
        itflights.append((row.departureCity, row.arrivalCity))
        ItineraryFlights[row.departureCity, row.arrivalCity] = itflights
    else:
        itflights.append((row.departureCity, row.Hub))
        itflights.append((row.Hub, row.arrivalCity))
        ItineraryFlights[row.departureCity, row.arrivalCity] = itflights
        
Demand = {}
Distances = {}
for index,row in merged_df.iterrows():
    Demand[row.departureCity, row.arrivalCity] = row.Demand
    Distances[row.departureCity, row.arrivalCity, "Direct"] = row.DirectDistance
    if row.Hub != "":
        Distances[row.departureCity, row.Hub, row.arrivalCity, "Indirect"] = row.IndirectDistance
        
IndirectUsed = {}
for index,row in merged_df.iterrows():
    IndirectUsed[row.departureCity, row.arrivalCity] = row.IndirectUsed
    
HubCities = list(set(merged_df.Hub.unique()))
HubCities = HubCities[1:]

# Make a dictionary of all the demands for the destinations
demand = merged_df.set_index(['departureCity', 'arrivalCity'])['Demand'].to_dict()
indirect_demand = merged_df.set_index(['departureCity', 'Hub', 'arrivalCity'])['Demand'].to_dict()


# Make a dictionary of all the distances for the destinations
indirect_df=merged_df[merged_df['Hub']!='']
indirect_distance = indirect_df.set_index(['departureCity', 'arrivalCity'])[['IndirectDistance']].to_dict()
indirect_distances = indirect_distance['IndirectDistance']
direct_distances = merged_df.set_index(['departureCity', 'arrivalCity'])['DirectDistance'].to_dict()
indirect_routes = list(indirect_distances.keys())
direct_routes = list(direct_distances.keys())

route_distance = merged_df.set_index(['departureCity', 'Hub', 'arrivalCity'])['RouteDistance'].to_dict()
route_list = list(route_distance.keys())

In [15]:
charge_indirect = 0.08
charge_direct = 0.1
cost_per_mile = {"small":4.5, "medium":8, "large":20}
capacity = {"small":50, "medium":100, "large":300}

model_test = gb.Model("RevenueOptimization")

# Variables the number of each size of plane that is going to fly
small = model_test.addVars(ItineraryFlights.keys(), name = "small", vtype = gb.GRB.INTEGER, lb = 0)
medium = model_test.addVars(ItineraryFlights.keys(), name = "medium", vtype = gb.GRB.INTEGER, lb = 0)
large = model_test.addVars(ItineraryFlights.keys(), name = "large", vtype = gb.GRB.INTEGER, lb = 0)

direct_passengers = model_test.addVars(ItineraryFlights.keys(), name = "direct_pass", vtype = gb.GRB.INTEGER, lb = 0)
indirect_passengers = model_test.addVars(ItineraryFlights, name = "indirect_pass", vtype = gb.GRB.INTEGER, lb = 0)

hub = model_test.addVars(HubCities, name = "hub", vtype = gb.GRB.BINARY, lb = 0)

M = 1000000000000

model_test.setObjective(gb.quicksum(charge_direct * direct_distances[route] * direct_passengers[route] -
                                    direct_distances[route] * (cost_per_mile["small"] * small[route] +
                                                               cost_per_mile["medium"] * medium[route] +
                                                               cost_per_mile["large"] * large[route])
                                    for route in ItineraryFlights.keys()) +
                        gb.quicksum(charge_indirect * direct_distances[route] * indirect_passengers[route] -
                                    indirect_distances.get(route, 0) * (cost_per_mile["small"] * small[route] +
                                                                        cost_per_mile["medium"] * medium[route] +
                                                                        cost_per_mile["large"] * large[route])
                                    for route in ItineraryFlights) ,gb.GRB.MAXIMIZE)

# Objective Function
# Constraints

model_test.addConstrs((direct_passengers[route]) <= Demand[route]
                     for route in  ItineraryFlights.keys())
model_test.addConstrs((indirect_passengers[route]) <= Demand[route]
                     for route in ItineraryFlights.keys())
    
model_test.addConstrs((direct_passengers[route] + indirect_passengers[route]) <= Demand[route] 
                      for route in ItineraryFlights.keys())

model_test.addConstrs((direct_passengers[route] + indirect_passengers[route]) <= capacity["small"] * small[route] + capacity["medium"] * medium[route] + capacity["large"] * large[route]
                      for route in ItineraryFlights.keys())

for city in HubCities:
    model_test.addConstr(gb.quicksum(indirect_passengers[route] for route in ItineraryFlights.keys() if city in route) <= hub[city] * M)
model_test.addConstr(gb.quicksum(hub[k] for k in HubCities) == 2)

model_test.optimize()

if model_test.status == gb.GRB.OPTIMAL:

    selected_hubs = [city for city in HubCities if hub[city].x > 0.5]
    print("Selected Hubs:", selected_hubs)
else:
    print("Optimization was not successful.")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6602 rows, 8241 columns and 18122 nonzeros
Model fingerprint: 0x3d80cc66
Variable types: 0 continuous, 8241 integer (41 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+12]
  Objective range  [2e+00, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective -0.0000000
Presolve removed 6601 rows and 8237 columns
Presolve time: 0.48s
Presolved: 1 rows, 4 columns, 4 nonzeros
Found heuristic solution: objective 165652.40000
Variable types: 0 continuous, 4 integer (1 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.51 seconds (0.02 work units)
Thread count was 8

In [30]:
vals = model_test.printAttr(["X", "Obj"])
all_vars = model_test.getVars()

values = model_test.getAttr("X", all_vars)
names = model_test.getAttr("VarName", all_vars)

res = pd.DataFrame({"names": names, "values": values})

daily_profit = model_test.ObjVal

small_res = res[res["names"].str.startswith("small")]["values"].sum()
medium_res = res[res["names"].str.startswith("medium")]["values"].sum()
large_res = res[res["names"].str.startswith("large")]["values"].sum()



    Variable            X          Obj 
--------------------------------------
small[Madrid,London]            0      -7429.5 
small[Paris,London]           -0       -958.5 
small[Frankfurt,London]            0      -4081.5 
small[Amsterdam,London]            1         -999 
small[Istanbul,London]            0       -14130 
small[Palma de Mallorca,London]            0      -7600.5 
small[Rome,London]            0        -8100 
small[Moscow,London]            0     -15331.5 
small[Lisbon,London]            0        -8892 
small[Zurich,London]            0      -4666.5 
small[Milan,London]            0        -5463 
small[Copenhagen,London]            0        -5508 
small[Manchester,London]            0        -1503 
small[Malaga,London]            0       -10089 
small[Vienna,London]            0        -7290 
small[Oslo,London]            0        -6948 
small[Dusseldorf,London]            0        -2907 
small[Dublin,London]            0      -2776.5 
small[Athens,London]           


KeyboardInterrupt

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/ipykernel/iostream.py", line 564, in flush
    if not evt.wait(self.flush_timeout):
  File "/opt/anaconda3/lib/python3.8/threading.py", line 558, in wait
    signaled = self._cond.wait(timeout)
  File "/opt/anaconda3/lib/python3.8/threading.py", line 306, in wait
    gotit = waiter.acquire(True, timeout)
KeyboardInterrupt: 


medium[Trondheim,Birmingham]           -0       -14624 
medium[London,Milan/Bergamo]            0        -9720 
medium[Madrid,Milan/Bergamo]            0       -13368 
medium[Paris,Milan/Bergamo]            0        -6728 
medium[Frankfurt,Milan/Bergamo]            0        -5024 
medium[Amsterdam,Milan/Bergamo]            0        -8168 
medium[Istanbul,Milan/Bergamo]            0       -18000 
medium[Palma de Mallorca,Milan/Bergamo]            0        -8824 
medium[Rome,Milan/Bergamo]            0        -4944 
medium[Moscow,Milan/Bergamo]            0       -23128 
medium[Lisbon,Milan/Bergamo]            0       -18208 
medium[Zurich,Milan/Bergamo]            0        -2328 
medium[Milan,Milan/Bergamo]            0         -216 
medium[Copenhagen,Milan/Bergamo]            0       -11456 
medium[Manchester,Milan/Bergamo]            0       -12176 
medium[Malaga,Milan/Bergamo]            0       -16600 
medium[Vienna,Milan/Bergamo]            0        -6216 
medium[Oslo,Milan/Bergamo


KeyboardInterrupt

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/ipykernel/iostream.py", line 564, in flush
    if not evt.wait(self.flush_timeout):
  File "/opt/anaconda3/lib/python3.8/threading.py", line 558, in wait
    signaled = self._cond.wait(timeout)
  File "/opt/anaconda3/lib/python3.8/threading.py", line 306, in wait
    gotit = waiter.acquire(True, timeout)
KeyboardInterrupt: 


large[Hamburg,London]            0       -19160 
large[Warsaw,London]            0       -40680 
large[Budapest,London]            0       -37540 
large[Geneva,London]            0       -18640 
large[Kyiv,London]            0       -56620 
large[Bucharest,London]            0       -53060 
large[Birmingham,London]            1        -2020 
large[Milan/Bergamo,London]            0       -24300 
large[Edinburgh,London]            0       -13400 
large[Nice,London]            0       -27660 
large[Cologne/Bonn,London]            0       -12920 
large[Saint Petersburg,London]            0       -57040 
large[Stuttgart,London]            0       -20740 
large[Lyon,London]            0       -18260 
large[Trondheim,London]            0       -37420 
large[London,Madrid]            0       -33020 
large[Paris,Madrid]            0       -29280 
large[Frankfurt,Madrid]            0       -37260 
large[Amsterdam,Madrid]            0       -41520 
large[Istanbul,Madrid]            0       -7162

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/ipykernel/iostream.py", line 564, in flush
    if not evt.wait(self.flush_timeout):
  File "/opt/anaconda3/lib/python3.8/threading.py", line 558, in wait
    signaled = self._cond.wait(timeout)
  File "/opt/anaconda3/lib/python3.8/threading.py", line 306, in wait
    gotit = waiter.acquire(True, timeout)
KeyboardInterrupt: 


large[Dusseldorf,Istanbul]            0       -52960 
large[Dublin,Istanbul]            0       -74220 
large[Athens,Istanbul]            1        -6980 
large[Porto,Istanbul]            0       -79940 
large[Helsinki,Istanbul]            0       -55300 
large[Stockholm,Istanbul]            0       -54300 
large[Berlin,Istanbul]            0       -45280 
large[Prague,Istanbul]            0       -42420 
large[Hamburg,Istanbul]            0       -55340 
large[Warsaw,Istanbul]            0       -38920 
large[Budapest,Istanbul]            0       -26820 
large[Geneva,Istanbul]            0       -48680 
large[Kyiv,Istanbul]            0       -27940 
large[Bucharest,Istanbul]            0        -5540 
large[Birmingham,Istanbul]            0       -66640 
large[Milan/Bergamo,Istanbul]            0       -45000 
large[Edinburgh,Istanbul]            0       -73060 
large[Nice,Istanbul]            0       -49680 
large[Cologne/Bonn,Istanbul]            0       -51900 
large[Saint Petersbu